In [1]:
from src.image_filter import ImageFilter
from src.visualizer import Visualizer
from src.camera import Camera
from src.lane_finder import LaneFinder

import numpy as np
import pickle
import glob
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
calibration_data = pickle.load(open("calibration_data.p", "rb" ))
calibration_data

{'camera_matrix': array([[1.15777818e+03, 0.00000000e+00, 6.67113856e+02],
        [0.00000000e+00, 1.15282217e+03, 3.86124584e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'distortion_coefficient': array([[-0.24688507, -0.02373153, -0.00109831,  0.00035107, -0.00259872]])}

In [3]:
prespective_data = pickle.load(open("prespective_data.p", "rb" ))
prespective_data

{'car_points': [(580, 460), (205, 720), (1110, 720), (703, 460)],
 'bird_points': [(320, 0), (320, 720), (960, 720), (960, 0)]}

In [4]:
camera = Camera(calibration_data, prespective_data['car_points'], prespective_data['bird_points'])
visualizer = Visualizer()

In [5]:
filter_dict = {
    'color1_min_threshold':40,        'color1_max_threshold':205,
    'color2_min_threshold':120,        'color2_max_threshold':255,
    'magnitude_min_threshold':20,
    'absolute_min_threshold' : 40,
    'direction_min_threshold':0.7,    'direction_max_threshold':1.4,
    'kernal_size':3
}
image_filter = ImageFilter(threshold_dict=filter_dict)

In [6]:
lane_finder_dict = {
    'n_window':9,
    'margin':100,
    'minpix':50,
    'ym_per_pix':30 / 720,
    'xm_per_pix':3.7 / 700
}
lane_finder = LaneFinder(lane_finder_dict)

In [22]:
def pipeline(image):
    undist = camera.get_undistort_image(image)
    filtered_image, _ = image_filter.get_binary_image(undist)
    bird_view = camera.get_bird_view(filtered_image)
    
    #result = lane_finder.process_video(bird_view)
    result = lane_finder.process_video(bird_view)

    font = cv2.FONT_HERSHEY_SIMPLEX

    projected_image = camera.get_projected_image(undist, bird_view, result['ploty'], result['leftx'], result['rightx'])

    # write right, left lane curvature
    cv2.putText(projected_image,'left curvature {} m'.format(result['left_radius']),(10,40), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(projected_image,'right curvature {} m'.format(result['right_radius']),(10,80), font, 1,(255,255,255),2,cv2.LINE_AA)
    # write offset from the center
    cv2.putText(projected_image,'off center by {} m'.format(result['vehicle_position']),(10,120), font, 1,(255,255,255),2,cv2.LINE_AA)
    return projected_image


In [8]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [23]:
project_video_output = '../test_videos_output/project_video.mp4'
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("../project_video.mp4")
project_video = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time project_video.write_videofile(project_video_output, audio=False)
lane_finder.reset()

|███████▊  | 987/1260 [03:51<01:05,  4.15it/s, now=None]

t:  78%|███████▊  | 988/1260 [03:51<01:05,  4.14it/s, now=None]

t:  78%|███████▊  | 989/1260 [03:52<01:05,  4.15it/s, now=None]

t:  79%|███████▊  | 990/1260 [03:52<01:03,  4.28it/s, now=None]

t:  79%|███████▊  | 991/1260 [03:52<01:03,  4.24it/s, now=None]

t:  79%|███████▊  | 992/1260 [03:52<01:02,  4.29it/s, now=None]

t:  79%|███████▉  | 993/1260 [03:53<01:02,  4.26it/s, now=None]

t:  79%|███████▉  | 994/1260 [03:53<01:01,  4.33it/s, now=None]

t:  79%|███████▉  | 995/1260 [03:53<01:02,  4.26it/s, now=None]

t:  79%|███████▉  | 996/1260 [03:53<01:01,  4.31it/s, now=None]

t:  79%|███████▉  | 997/1260 [03:53<01:00,  4.32it/s, now=None]

t:  79%|███████▉  | 998/1260 [03:54<01:01,  4.28it/s, now=None]

t:  79%|███████▉  | 999/1260 [03:54<01:00,  4.33it/s, now=None]

t:  79%|███████▉  | 1000/1260 [03:54<01:00,  4.27it/s, now=None]

t:  79%|███████▉  | 1001/1260 [03:54<01:00,  4.26it/s, now=None]

t:  80%|███████▉  | 1002/1260 

In [19]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video_Clip))

In [21]:
challenge_video_output = '../test_videos_output/challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip2 = VideoFileClip("../challenge_video.mp4")
challenge_video = clip2.fl_image(pipeline) #NOTE: this function expects color images!!
%time challenge_video.write_videofile(challenge_video_output, audio=False)
lane_finder.reset()

t:   0%|          | 0/485 [00:00<?, ?it/s, now=None]Moviepy - Building video ../test_videos_output/challenge_video.mp4.
Moviepy - Writing video ../test_videos_output/challenge_video.mp4


t:  28%|██▊       | 138/485 [00:29<01:14,  4.65it/s, now=None]

TypeError: expected non-empty vector for x

In [17]:
harder_challenge_video_output = '../test_videos_output/harder_challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip3 = VideoFileClip("../harder_challenge_video.mp4")
harder_challenge_video = clip3.fl_image(pipeline) #NOTE: this function expects color images!!
%time harder_challenge_video.write_videofile(harder_challenge_video_output, audio=False)
lane_finder.reset()

t:   0%|          | 0/125 [00:00<?, ?it/s, now=None]Moviepy - Building video ../test_videos_output/harder_challenge_video.mp4.
Moviepy - Writing video ../test_videos_output/harder_challenge_video.mp4


t:  71%|███████▏  | 107/150 [02:11<00:08,  5.16it/s, now=None]Moviepy - Done !
Moviepy - video ready ../test_videos_output/harder_challenge_video.mp4
Wall time: 27.3 s
